In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [8]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [9]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [10]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [11]:
neighborhoods_data = newyork_data['features']

In [12]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [14]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [15]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [16]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [17]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [19]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [15]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [20]:
# Here one can select the borough in which they are planning of opening their business
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [21]:
address = 'Queens, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [19]:
# we then create of tyhe map of the region the user wants to see 
# in this case its Queens

map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

In [22]:
CLIENT_ID = 'N3JZIOYSXFEDAYISCE3ZDMGEYXNBPIQNFUHPWPVEZPOWEQ3E' # your Foursquare ID
CLIENT_SECRET = 'PODRHWMWH4XCWWLQFUQZSVIVK3UDZ5MLI1Y0IWQKNTUBQYPJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N3JZIOYSXFEDAYISCE3ZDMGEYXNBPIQNFUHPWPVEZPOWEQ3E
CLIENT_SECRET:PODRHWMWH4XCWWLQFUQZSVIVK3UDZ5MLI1Y0IWQKNTUBQYPJ


## NOW WE EXPLORE THE NEIGHBOURHOODS

In [23]:
# now one can choose te neighbourhood in which they want to start a business
queens_data.loc[2, 'Neighborhood']

'Jackson Heights'

In [24]:
neighborhood_latitude = queens_data.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = queens_data.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = queens_data.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Jackson Heights are 40.75198138007367, -73.88282109164365.


In [25]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=N3JZIOYSXFEDAYISCE3ZDMGEYXNBPIQNFUHPWPVEZPOWEQ3E&client_secret=PODRHWMWH4XCWWLQFUQZSVIVK3UDZ5MLI1Y0IWQKNTUBQYPJ&v=20180605&ll=40.75198138007367,-73.88282109164365&radius=500&limit=100'

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed14b159fcb92001b0b91b0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Jackson Heights',
  'headerFullLocation': 'Jackson Heights, Queens',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 81,
  'suggestedBounds': {'ne': {'lat': 40.756481384573675,
    'lng': -73.87689191538273},
   'sw': {'lat': 40.74748137557367, 'lng': -73.88875026790457}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '52dac66c498e40890ad671e7',
       'name': 'Downtown Natural Market',
       'location': {'address': '84-01 37th Ave',
        'lat': 40.750045874212105,
        'lng': -73.88240697837537,
        'l

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Natural Market,Grocery Store,40.750046,-73.882407
1,Oceanic Boil,Cajun / Creole Restaurant,40.749952,-73.881903
2,La Nueva Bakery,South American Restaurant,40.750134,-73.880227
3,La Picada Azuaya,Spanish Restaurant,40.750140,-73.881775
4,Pollos a la Brasa Mario,Latin American Restaurant,40.749806,-73.883243


###  RESULT :As we see that there are many restaurants in this region and hence buying a place here to open a restaurant may not lead to a good profit 

### OBSERVATION: However if we plan on opening an Indian Restaurant here , it might just workout 

In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

81 venues were returned by Foursquare.


## NOW WE EXPLORE NEIGHBOURHOODS IN QUEENS

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
# NOW WE GET ALL THE VENUES IN ALL NEIGHBOURHOODS OF QUEENS 

queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )


Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


In [33]:
# we chwck the size of our dataframe
print(queens_venues.shape)
queens_venues.head(50)

(2083, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant
5,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym
6,Astoria,40.768509,-73.915654,Sitan Muay Thai,40.766108,-73.913224,Martial Arts Dojo
7,Astoria,40.768509,-73.915654,Al-sham Sweets and Pastries,40.768077,-73.911561,Middle Eastern Restaurant
8,Astoria,40.768509,-73.915654,SVL Bar,40.769816,-73.918530,Greek Restaurant
9,Astoria,40.768509,-73.915654,Zyara Restaurant,40.766591,-73.912713,Restaurant


In [34]:
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,18,18,18,18,18,18
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,18,18,18,18,18,18
Bay Terrace,37,37,37,37,37,37
Bayside,75,75,75,75,75,75
Bayswater,2,2,2,2,2,2
Beechhurst,16,16,16,16,16,16
Bellaire,13,13,13,13,13,13


In [35]:
#now we look for the unique categories in our dataframe 
print('There are {} uniques categories.'.format(len(queens_venues['Venue Category'].unique())))

There are 273 uniques categories.


###  ANALYSE EACH NEIGHBOURHOOD

In [36]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[-1]] + list(queens_onehot.columns[:-1])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wi

In [37]:
queens_onehot.shape

(2083, 273)

### NOW WE GROUP THE NEIGHBOURHOODS TAKING MEAN FREQUENCY OCCURANCE OF EACH CATEGORY

In [38]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wi

### So regions that have a frequency of 0 in any category of shops your interested in opening 
### One can think of opening a shop 

In [39]:
#Now we chweck the size again
queens_grouped.shape

(81, 273)

### NOW WE PRINT EACH NEIGHBOURHOOD WITH TOP 6 MOST COMMON VENUES 

In [40]:
num_top_venues = 6

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
             venue  freq
0        Surf Spot  0.22
1   Sandwich Place  0.11
2    Metro Station  0.11
3            Beach  0.06
4        Wine Shop  0.06
5  Bed & Breakfast  0.06


----Astoria----
                       venue  freq
0                        Bar  0.06
1  Middle Eastern Restaurant  0.06
2           Greek Restaurant  0.04
3   Mediterranean Restaurant  0.04
4          Indian Restaurant  0.04
5                 Hookah Bar  0.04


----Astoria Heights----
           venue  freq
0  Deli / Bodega  0.08
1     Playground  0.08
2    Bus Station  0.08
3  Bowling Alley  0.08
4    Supermarket  0.08
5   Cocktail Bar  0.08


----Auburndale----
                venue  freq
0  Athletics & Sports  0.06
1  Miscellaneous Shop  0.06
2   Mobile Phone Shop  0.06
3    Toy / Game Store  0.06
4           Pet Store  0.06
5            Pharmacy  0.06


----Bay Terrace----
                 venue  freq
0       Clothing Store  0.11
1        Women's Store  0.05
2       Cosmetics Shop  0.05
3   

               venue  freq
0        Bus Station   0.2
1  Indian Restaurant   0.2
2       Intersection   0.2
3       Antique Shop   0.2
4             Lounge   0.2
5       Optical Shop   0.0


----Jamaica Hills----
                  venue  freq
0   Fried Chicken Joint  0.07
1     Indian Restaurant  0.07
2  Fast Food Restaurant  0.07
3              Pharmacy  0.07
4            Donut Shop  0.07
5    Chinese Restaurant  0.07


----Kew Gardens----
                venue  freq
0  Chinese Restaurant  0.07
1      Cosmetics Shop  0.05
2                 Bar  0.05
3                Bank  0.05
4                Park  0.05
5   Indian Restaurant  0.05


----Kew Gardens Hills----
              venue  freq
0              Bank  0.14
1        Restaurant  0.09
2        Playground  0.09
3       Bus Station  0.09
4       Pizza Place  0.09
5  Sushi Restaurant  0.05


----Laurelton----
                  venue  freq
0  Caribbean Restaurant  0.44
1        Cosmetics Shop  0.22
2         Train Station  0.11
3        

                  venue  freq
0   Rental Car Location  0.14
1         Women's Store  0.05
2                Bakery  0.05
3    Italian Restaurant  0.05
4   Japanese Restaurant  0.05
5  Gym / Fitness Center  0.05


----Sunnyside----
                venue  freq
0         Pizza Place  0.12
1  Chinese Restaurant  0.07
2  Italian Restaurant  0.05
3              Bakery  0.05
4       Deli / Bodega  0.05
5         Coffee Shop  0.05


----Sunnyside Gardens----
                venue  freq
0                 Bar  0.07
1       Grocery Store  0.06
2         Pizza Place  0.04
3  Turkish Restaurant  0.03
4            Pharmacy  0.03
5         Coffee Shop  0.03


----Utopia----
                       venue  freq
0              Deli / Bodega  0.14
1            Automotive Shop  0.07
2                 Playground  0.07
3                Pizza Place  0.07
4                 Donut Shop  0.07
5  South American Restaurant  0.07


----Whitestone----
             venue  freq
0      Candy Store   0.2
1  Bubble Tea Sho

In [41]:
# now we put that into pandas dataframe 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now we display 10 most common visited venues in each neighbourhood , this information can help the individual or company to know about what the people in each neighbourhood like to visit and can properly also predict if the store or business idea will work in that neighbourhood!!!!


In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Sandwich Place,Metro Station,Board Shop,Beach,Thai Restaurant,Donut Shop,Bed & Breakfast,Coffee Shop,Playground
1,Astoria,Bar,Middle Eastern Restaurant,Indian Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Café,Bakery,Seafood Restaurant,Deli / Bodega
2,Astoria Heights,Hostel,Bakery,Pizza Place,Deli / Bodega,Cocktail Bar,Bus Station,Burger Joint,Playground,Bowling Alley,Supermarket
3,Auburndale,Hookah Bar,Train,Sushi Restaurant,Supermarket,Noodle House,Bar,Furniture / Home Store,Italian Restaurant,Athletics & Sports,Toy / Game Store
4,Bay Terrace,Clothing Store,Women's Store,Kids Store,Lingerie Store,Shoe Store,Donut Shop,Mobile Phone Shop,American Restaurant,Cosmetics Shop,Coffee Shop


### NOW WE CLUSTER THE NEIGHBOURHOODS

In [63]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [65]:
queens_grouped_clustering.insert(0, 'Cluster Labels', kmeans.labels_)

queens_grouped_clustering = queens_data                                        
queens_grouped_clustering = queens_grouped_clustering.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood') 
queens_grouped_clustering.head(10)

,Cluster Labels,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Queens,Astoria,40.768509,-73.915654,Bar,Middle Eastern Restaurant,Indian Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Café,Bakery,Seafood Restaurant,Deli / Bodega
1,0,Queens,Woodside,40.746349,-73.901842,Grocery Store,Thai Restaurant,Bakery,Bar,Filipino Restaurant,Latin American Restaurant,Donut Shop,American Restaurant,Pub,Deli / Bodega
2,0,Queens,Jackson Heights,40.751981,-73.882821,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Mexican Restaurant,Grocery Store,Thai Restaurant,Pizza Place,Spanish Restaurant
3,0,Queens,Elmhurst,40.744049,-73.881656,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,South American Restaurant,Vietnamese Restaurant,Colombian Restaurant,Snack Place,Food Court,Sushi Restaurant,Bar
4,0,Queens,Howard Beach,40.654225,-73.838138,Pharmacy,Italian Restaurant,Sandwich Place,Bagel Shop,Fast Food Restaurant,Seafood Restaurant,Supermarket,Sporting Goods Shop,Convenience Store,Mexican Restaurant
5,0,Queens,Corona,40.742382,-73.856825,Mexican Restaurant,Italian Restaurant,Sandwich Place,Donut Shop,Supermarket,South American Restaurant,Bakery,Empanada Restaurant,Restaurant,Park
6,1,Queens,Forest Hills,40.725264,-73.844475,Gym / Fitness Center,Gym,Yoga Studio,Pharmacy,Pizza Place,Park,Thai Restaurant,Convenience Store,Video Game Store,Deli / Bodega
7,0,Queens,Kew Gardens,40.705179,-73.829819,Chinese Restaurant,Cosmetics Shop,Pharmacy,Bar,Indian Restaurant,Pizza Place,Bank,Park,Donut Shop,Food
8,0,Queens,Richmond Hill,40.697947,-73.831833,Bank,Pizza Place,Latin American Restaurant,Lounge,Diner,Chinese Restaurant,Café,Caribbean Restaurant,Clothing Store,Discount Store
9,0,Queens,Flushing,40.764454,-73.831773,Hotpot Restaurant,Chinese Restaurant,Bakery,Bubble Tea Shop,Szechuan Restaurant,Construction & Landscaping,Gym / Fitness Center,Karaoke Bar,Korean Restaurant,Hotel


In [55]:
queens_merged.head()

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,Bar,Middle Eastern Restaurant,Indian Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Café,Bakery,Seafood Restaurant,Deli / Bodega
1,Queens,Woodside,40.746349,-73.901842,Grocery Store,Thai Restaurant,Bakery,Bar,Filipino Restaurant,Latin American Restaurant,Donut Shop,American Restaurant,Pub,Deli / Bodega
2,Queens,Jackson Heights,40.751981,-73.882821,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Mexican Restaurant,Grocery Store,Thai Restaurant,Pizza Place,Spanish Restaurant
3,Queens,Elmhurst,40.744049,-73.881656,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,South American Restaurant,Vietnamese Restaurant,Colombian Restaurant,Snack Place,Food Court,Sushi Restaurant,Bar
4,Queens,Howard Beach,40.654225,-73.838138,Pharmacy,Italian Restaurant,Sandwich Place,Bagel Shop,Fast Food Restaurant,Seafood Restaurant,Supermarket,Sporting Goods Shop,Convenience Store,Mexican Restaurant


In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        queens_grouped_clustering['Latitude'], 
        queens_grouped_clustering['Longitude'], 
        queens_grouped_clustering['Neighborhood'], 
        queens_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


## Now we examine clusters 

In [68]:
queens_grouped_clustering.loc[queens_grouped_clustering['Cluster Labels'] == 0, queens_grouped_clustering.columns[[1] + list(range(5, queens_grouped_clustering.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Bar,Middle Eastern Restaurant,Indian Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Café,Bakery,Seafood Restaurant,Deli / Bodega
1,Queens,Grocery Store,Thai Restaurant,Bakery,Bar,Filipino Restaurant,Latin American Restaurant,Donut Shop,American Restaurant,Pub,Deli / Bodega
2,Queens,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Mexican Restaurant,Grocery Store,Thai Restaurant,Pizza Place,Spanish Restaurant
3,Queens,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,South American Restaurant,Vietnamese Restaurant,Colombian Restaurant,Snack Place,Food Court,Sushi Restaurant,Bar
4,Queens,Pharmacy,Italian Restaurant,Sandwich Place,Bagel Shop,Fast Food Restaurant,Seafood Restaurant,Supermarket,Sporting Goods Shop,Convenience Store,Mexican Restaurant
5,Queens,Mexican Restaurant,Italian Restaurant,Sandwich Place,Donut Shop,Supermarket,South American Restaurant,Bakery,Empanada Restaurant,Restaurant,Park
7,Queens,Chinese Restaurant,Cosmetics Shop,Pharmacy,Bar,Indian Restaurant,Pizza Place,Bank,Park,Donut Shop,Food
8,Queens,Bank,Pizza Place,Latin American Restaurant,Lounge,Diner,Chinese Restaurant,Café,Caribbean Restaurant,Clothing Store,Discount Store
9,Queens,Hotpot Restaurant,Chinese Restaurant,Bakery,Bubble Tea Shop,Szechuan Restaurant,Construction & Landscaping,Gym / Fitness Center,Karaoke Bar,Korean Restaurant,Hotel
10,Queens,Hotel,Coffee Shop,Bar,Pizza Place,Mexican Restaurant,Café,Deli / Bodega,Gym / Fitness Center,Bus Station,Cocktail Bar


In [69]:
queens_grouped_clustering.loc[queens_grouped_clustering['Cluster Labels'] == 1, queens_grouped_clustering.columns[[1] + list(range(5, queens_grouped_clustering.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Queens,Gym / Fitness Center,Gym,Yoga Studio,Pharmacy,Pizza Place,Park,Thai Restaurant,Convenience Store,Video Game Store,Deli / Bodega
69,Queens,Deli / Bodega,Spa,History Museum,Playground,Basketball Court,Bakery,Automotive Shop,Donut Shop,Indie Movie Theater,Ice Cream Shop


In [70]:
queens_grouped_clustering.loc[queens_grouped_clustering['Cluster Labels'] == 2, queens_grouped_clustering.columns[[1] + list(range(5, queens_grouped_clustering.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Queens,Donut Shop,Hotel Bar,Coffee Shop,Bus Station,Supermarket,Gas Station,Electronics Store,Bakery,Lake,Rental Car Location
31,Queens,Mobile Phone Shop,Caribbean Restaurant,Shoe Store,Pizza Place,Performing Arts Venue,Department Store,Coffee Shop,Clothing Store,Mexican Restaurant,Sandwich Place


In [71]:
queens_grouped_clustering.loc[queens_grouped_clustering['Cluster Labels'] == 3, queens_grouped_clustering.columns[[1] + list(range(5, queens_grouped_clustering.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Queens,Deli / Bodega,Food & Drink Shop,Brewery,Chinese Restaurant,Pizza Place,Intersection,Arts & Crafts Store,Farm,Electronics Store,Empanada Restaurant


In [72]:
queens_grouped_clustering.loc[queens_grouped_clustering['Cluster Labels'] == 4, queens_grouped_clustering.columns[[1] + list(range(5, queens_grouped_clustering.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,Queens,Dosa Place,Basketball Court,Indian Restaurant,Grocery Store,Pizza Place,Chinese Restaurant,Falafel Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant


### Now reviewing the notebook , one can see for example :
### We have a client that wants to open an Indian Restaurant , as we can see that for cluster 4 , the most visited places consists of the Dosa Place and an Indian Restaurant , hence it might not be a good idea to open a restaurant there 

### However when we see cluster - 1 , we notice that for 6 and 69 , not many restaurants make it to the top 10 and hence these might be good places to open an Indian Restaurant

### On the contrary Cluster 0 doesnt seem like a good region to open a resturant in